# Earthquake Impact Analysis in Afghanistan (2022)

## Objective
This notebook aims to analyze and visualize the impact of earthquakes in Afghanistan during the year 2022. By integrating and examining various geospatial datasets, we seek to identify the populations most affected by these seismic events.

## Steps of the Analysis

1. **Data Retrieval:**
   - Fetching vector data from the eoAPI.
   - Listing available datasets related to earthquakes and population density in Afghanistan.

2. **Data Visualization:**
   - Adding hexbin and shakemap layers to an interactive map.
   - Styling the layers for better understanding and differentiation.

3. **Data Processing:**
   - Performing a spatial join using the centroids of hexagons to avoid duplicates.
   - Ensuring the accuracy and relevance of the merged data.

4. **In-depth Analysis:**
   - Visualizing the `paramvalue` attribute of the joined layer to assess earthquake intensity.
   - Analyzing the relationship between population density and earthquake intensity.
   
5. **Conclusion and Implications:**
   - Summarizing key findings and insights.
   - Discussing the implications for earthquake preparedness and response in Afghanistan.


## Step 0: Setup

Before diving into the analysis, it's important to set up our environment by importing all the necessary libraries. This includes libraries for handling HTTP requests, data manipulation, and geospatial analysis. Ensuring that all dependencies are correctly imported at the start helps in avoiding any interruptions in the workflow during the analysis.


In [2]:
import IPython
!python -m pip install httpx ipyleaflet matplotlib geopandas
IPython.display.clear_output(wait=False)

In [3]:
import httpx
import geopandas as gpd  
import pandas as pd
import httpx
import ipyleaflet 

# Additional libraries can be added as required for specific tasks


## Step 1: Data Retrieval

In this initial step, our focus is on fetching the relevant vector data from eoAPI. This involves accessing the ingested datasets that pertain to earthquakes and population density in Afghanistan.

We will be using eoAPI's vector endpoints to retrieve information on the vector datasets:

In [4]:
vector_endpoint = "https://eoapi.ifrc-risk.k8s.labs.ds.io/vector"

We can list the available datasets by querying the available collections:

In [12]:
response = httpx.get(f"{vector_endpoint}/collections").json()
collections = response["collections"]
for collection in collections:
    print(collection["id"])


public.my_data
public.population_hexbins_afghanistan
public.admin_boundaries_usa_adm0
public.admin_boundaries_usa_adm2
public.admin_boundaries_usa_adm1
public.health_facilities_afg_osm
public.earthquake_usgs_gov_shakemap_afg_pga
public.earthquake_usgs_gov_shakemap_afg_psa1p0
public.earthquake_usgs_gov_shakemap_afg_pgv
public.earthquake_usgs_gov_shakemap_afg_psa3p0
public.earthquake_usgs_gov_shakemap_afg_psa0p3
public.earthquake_usgs_gov_shakemap_afg_mi
public.st_subdivide
public.st_hexagongrid
public.st_squaregrid


To achieve our goal of identifying the populations most affected by the earthquakes in Afghanistan, we will utilize two key datasets from the eoAPI:

1. **Population Hexbins (`public.population_hexbins_afghanistan`):**
   This dataset provides detailed information about population density in Afghanistan. The data is structured in hexagonal bins, each representing a specific geographic area, allowing for a precise and comprehensive understanding of population distribution across the country.

2. **Earthquake Shakemap (`public.earthquake_usgs_gov_shakemap_afg_pga`):**
   The Earthquake Shakemap dataset offers crucial data on the intensity of earthquakes in Afghanistan. This information is vital for assessing the severity of the seismic events and their potential impact on different regions.

By integrating these datasets, we aim to create a comprehensive analysis that combines the spatial distribution of the population with the intensity of the earthquake events. This approach will enable us to effectively identify and visualize the areas and populations most affected by the earthquakes.

In [52]:
shake_data_url = f"{vector_endpoint}/collections/public.population_hexbins_afghanistan/items"
population_hexbins_url = f"{vector_endpoint}/collections/public.earthquake_usgs_gov_shakemap_afg_pga/items"

In [53]:
def get_all_items(url, limit=500):
    all_items = []
    
    while True:
        response = httpx.get(url, params={"limit": limit}, follow_redirects=True)
        if response.status_code == 200:
            items = response.json()
            all_items.extend(items.get("features", []))
            next_link = next((link["href"] for link in items.get("links", []) if link.get("rel") == "next"), None)
            if next_link:
                url = next_link
            else:
                break
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            break

    return all_items

In [55]:
shake_items = get_all_items(shake_data_url) # takes a few minutes


In [54]:
hexbin_items = get_all_items(population_hexbins_url)
